In [ ]:
%pip install transformers

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer

/home/cosmoquester/aibookathon/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [2]:
# 학습에 사용했던 요약문을 생성할 때와 똑같은 요약 모델과 생성 옵션을 사용하는 것이 좋습니다.
summarize_model_name: str = "digit82/kobart-summarization"
summarize_num_beams: int = 4
sum_input_max_length: int = 512
sum_max_token_length: int = 128

model_name: str = "khu-bot/polyglot-essayist-with-sum"
tokenizer_name: str = model_name
# Private으로 huggingface model hub에 올라가 있는 경우에 필요합니다.
auth_token: str = None
prompt_max_length: int = 128
max_length: int = 512
device: str = "cuda"

# 이 개수만큼의 글자를 생성하면 종료됩니다.
num_generate_characters: int = 2000
# 추론에 몇 개의 요약문을 사용할지를 명시합니다.
use_n_recent_summarizations: int = 10

# Load Model

In [3]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_auth_token=auth_token, **tokenizer_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=auth_token).to(device)

In [4]:
tokenizer_kwargs = {"padding_side": "left", "truncation_side": "left"}
summarize_tokenizer = AutoTokenizer.from_pretrained(summarize_model_name, use_auth_token=auth_token, **tokenizer_kwargs)
summarize_model = AutoModelForSeq2SeqLM.from_pretrained(summarize_model_name).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# Initial Condition

In [5]:
# 같은 title과 seed를 사용하면 결과를 재현할 수 있습니다. (물론 config도)
title: str = "나의 빛과 어둠"
seed: int = 42
backup_indices = [9, 5, 8, 0, 19, 0, 7, 14] # 기존에 title과 seed로 생성한 글의 selected indices를 넣으시면 같은 결과를 낼 수 있습니다.

# Write

In [6]:
import re

def datum_to_string(datum, use_n_summary: int = 10):
    text = f"제목: {datum['title']}\n"

    summarizations = datum.get("summarizations")
    if summarizations:
        summarizations = summarizations[-use_n_summary:]
        summarization = " ".join(summarizations).replace("\n", " ")
        text = f"요약: {summarization}\n" + text
    return text

def segment_text(text: str):
    """텍스트를 여러 개의 segment로 분리합니다."""
    seperate_indices = [m.end() for m in re.finditer(r"([.?!'\"”’]+(?!\w)\n*|\n+)\s*", text)]
    if len(text) not in seperate_indices:
        seperate_indices = seperate_indices + [len(text)]

    start_index = 0
    segments = []
    for end_index in seperate_indices:
        segments.append(text[start_index: end_index])
        start_index = end_index
    return segments

In [7]:
initial_input_example = {
    "title": title, 
    "summarizations": [], 
    "contents": []
}

In [8]:
from IPython.display import clear_output
from copy import deepcopy

torch.manual_seed(seed)
input_example = deepcopy(initial_input_example)
select_indices = []

while True:
  clear_output()
  print("Selected Indices:", select_indices)
  print("\n[TITLE]")
  print(input_example["title"])
  print("\n[SUMMARIZATIONS]")
  for i, summarization in enumerate(input_example["summarizations"], start=1):
    print(f"<{i}> {summarization}")
  print("\n[KEEP CONTENT]")
  keep_content = "".join(input_example["contents"])
  print(keep_content)
  print(f"\n[# OF CURRENT TEXT: {len(keep_content)}]")

  if len(keep_content) >= num_generate_characters:
    break

  prompt = datum_to_string(input_example)
  input_ids = tokenizer(
          [prompt],
          add_special_tokens=False,
          max_length=prompt_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  output = model.generate(input_ids, 
      max_length, 
      do_sample=True,
      num_return_sequences=1,
      pad_token_id=tokenizer.pad_token_id,
      use_cache=True,
  )
  output = output.squeeze(dim=0)[input_ids.size(1):]

  text = tokenizer.decode(output, skip_special_tokens=True)
  segments = segment_text(text)
  segments.insert(0, "")

  print("\n[CUR SENTENCES]")
  for i, segment in enumerate(segments):
    print(f"[{i}] {segment}")
  print("[-1] ※ EXIT: 더 이상 문장을 생성하지 않습니다.")

  if backup_indices:
    select_index = backup_indices.pop(0)
  else:
    while True:
      try:
        select_index = int(input("Select last sentence index: "))
        break
      except:
        print("숫자를 입력해주세요. 그만 생성하시려면 -1을 입력해주세요.")
        continue

  select_indices.append(select_index)
  if select_index < 0:
    break

  selected_segment = segments[:select_index + 1]
  selected_text = "".join(selected_segment)

  if not selected_text:
    continue

  sum_input_text = summarize_tokenizer.bos_token + selected_text + summarize_tokenizer.eos_token
  input_example["contents"].append(selected_text)
  sum_input_ids = summarize_tokenizer(
          [selected_text],
          add_special_tokens=False,
          max_length=sum_input_max_length,
          padding="longest",
          truncation=True,
          return_tensors="pt",
          return_token_type_ids=False,
          return_attention_mask=False,
  )["input_ids"].to(device)

  sum_outputs = summarize_model.generate(sum_input_ids, 
      sum_max_token_length, 
      num_beams=summarize_num_beams,
      num_return_sequences=1,
      eos_token_id=summarize_tokenizer.eos_token_id,
      pad_token_id=summarize_tokenizer.pad_token_id,
      use_cache=True,
  )

  summarization = summarize_tokenizer.decode(sum_outputs.squeeze(dim=0), skip_special_tokens=True)
  input_example["summarizations"].append(summarization)

Selected Indices: [9, 5, 8, 0, 19, 0, 7, 14]

[TITLE]
나의 빛과 어둠

[SUMMARIZATIONS]
<1> 대학 실기 시험, 대학 입학시험, 대학 수업 그리고 고3 수련회뿐인 나의 삶과 시가 온통 어둡고 칙칙한 색깔인 이유는 당연하다.
<2> 순수했고 때 묻지 않은 순수하고 때 묻지 않은 아이가 참 마음에 들었다.
<3> 밝고 따뜻한 사람이 되는 것을, 타인의 삶에도 마음을 내어주는 사람이 되는 것을 참 좋은 것이라고 생각하기 시작한 지 이미 오래고, 여전히 그런 것을 꿈꾸고, 여전히 그런 이들을 부러워하고, 그렇게 되지 못할까 봐 슬픈 것입니다.
<4> 내가 가진 빛과 어둠을 인정하는 게 쉬운 일은 아니지만, 인정할 것을 인정하자 그 다음부터가 쉬워진다.
<5> 내가 갖고 있는 좋은 것의 목록을 헤아려보려면, 나는 그 목록의 마지막을, 그 영역 밖의 무엇이 마지막 칸을 채울 수 있을 것 같다고 생각했던 것에 대해 말한다면, 나는 그 목록의 마지막을, 그 영역 밖의 무엇이 마지막 칸을 채울 수 있을 것 같다.
<6> 너무나 사소하고 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들을, 너무나 당연한 모든 것들은, 너무나 쉽게 잊거나 망각하는것들은, 쉽게 만들고, 쉬운 것으로부터 더 쉽게 벗어나고 더 쉽게 벗어나고 쉽게 잊고 잊고 잊고

[KEEP CONTENT]
나의 빛과 어둠
"내 나이 열여섯 살에 나는 너무 어렸다."
시인의 말처럼 나야말로 열여섯 살에 시를 쓸 수 있을 거라곤 전혀 생각

In [9]:
print("Initial Input Example:", initial_input_example)
print("Seed:", seed)
print("Backup Indices:", select_indices)
print("※ 위 정보를 이용하면 똑같은 결과를 다시 재현할 수 있습니다.")

Initial Input Example: {'title': '나의 빛과 어둠', 'summarizations': [], 'contents': []}
Seed: 42
Backup Indices: [9, 5, 8, 0, 19, 0, 7, 14]
※ 위 정보를 이용하면 똑같은 결과를 다시 재현할 수 있습니다.


In [10]:
print("※ 혹시나 중간과정을 스킵하고 이어서 추론을 원한다면 현재 `input_example`을 복원하고 seed를 아래의 값으로 설정하세요.")
print("Current Seed:", torch.seed())

※ 혹시나 중간과정을 스킵하고 이어서 추론을 원한다면 현재 `input_example`을 복원하고 seed를 아래의 값으로 설정하세요.
Current Seed: 11828976477152541601


In [12]:
import json
import os

output_path = title + ".json"

if os.path.exists(output_path):
    raise ValueError("Already existing output path!")

with open(output_path, "w") as f:
    json.dump({
        "config": {
            "summarize_model_name": summarize_model_name,
            "summarize_num_beams": summarize_num_beams,
            "sum_input_max_length": sum_input_max_length,
            "sum_max_token_length": sum_max_token_length,
            "model_name": model_name,
            "tokenizer_name": tokenizer_name,
            "prompt_max_length": prompt_max_length,
            "max_length": max_length,
            "device": device,
            "auth_token": auth_token,
            "num_generate_characters": num_generate_characters,
            "use_n_recent_summarizations": use_n_recent_summarizations,
        },
        "initial": {
            "example": initial_input_example,
            "seed": seed,
        },
        "output": {
            "selected_indices": select_indices,
            "example": input_example,
        }
    }, f, ensure_ascii=False, indent=2)